In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 5.3 MB/s 
     |████████████████████████████████| 170 kB 29.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=81951c4ef23298fa1903bdcb7bf332aee040160672856311e9fa3a7094ad1ed1
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 743 kB 5.0 MB/s 


In [ ]:
!pip install -U 'scikit-learn<0.24'

     |████████████████████████████████| 6.8 MB 5.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [ ]:
from sklearn.model_selection import train_test_split
import stanza
from collections import defaultdict
from tqdm.notebook import tqdm
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import cross_val_score
from collections import Counter
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,pos')

2021-12-26 21:21:59 INFO: Downloading default packages for language: ru (Russian)...


2021-12-26 21:22:16 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-12-26 21:22:16 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-12-26 21:22:16 INFO: Use device: gpu
2021-12-26 21:22:16 INFO: Loading: tokenize
2021-12-26 21:22:27 INFO: Loading: pos
2021-12-26 21:22:28 INFO: Done loading processors!


### Готовим файлы

In [ ]:
texts, ids = [], []
with open('/content/drive/MyDrive/train_reviews.txt') as f:
  for line in f:
    text_id, text = line.rstrip('\r\n').split('\t')
    texts.append(text)
    ids.append(text_id)

In [ ]:
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)

In [ ]:
train_aspects, dev_aspects = [], []
with open("/content/drive/MyDrive/train_aspects.txt") as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    if text_id in train_ids:
      train_aspects.append(line)
    if text_id in dev_ids:
      dev_aspects.append(line)

In [ ]:
train_sentiment, dev_sentiment = [], []
with open('/content/drive/MyDrive/train_cats.txt') as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    if text_id in train_ids:
      train_sentiment.append(line)
    if text_id in dev_ids:
      dev_sentiment.append(line)

In [ ]:
with open("/content/drive/MyDrive/train_split_aspects.txt", 'w') as f:
  for l in train_aspects:
    print(l, file=f)
with open('/content/drive/MyDrive/dev_aspects.txt', "w") as f:
  for l in dev_aspects:
    print(l, file=f)
with open('/content/drive/MyDrive/train_split_reviews.txt', "w") as f:
  for i, l in zip(train_ids, train_texts):
    print(i, l, sep="\t", file=f)
with open('/content/drive/MyDrive/dev_reviews.txt', "w") as f:
  for i, l in zip(dev_ids, dev_texts):
    print(i, l, sep="\t", file=f)
with open('/content/drive/MyDrive/train_split_cats.txt', "w") as f:
  for l in train_sentiment:
    print(l, file=f)
with open('/content/drive/MyDrive/dev_cats.txt', "w") as f:
  for l in dev_sentiment:
    print(l, file=f)

### Читаем файлы, с которыми будем работать в дальнейшем

In [ ]:
train_aspects = []
with open("/content/drive/MyDrive/more_train_split_aspects.txt") as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    train_aspects.append(line)
  

In [ ]:
dev_aspects = []
with open("/content/drive/MyDrive/dev_aspects.txt") as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    dev_aspects.append(line)

In [ ]:
dict_words = {}
for aspect in dev_aspects:
  cat = aspect.split('\t')[1]
  word = aspect.split('\t')[2]
  dict_words[word] = cat

In [ ]:
dict_sents = {}
for aspect in dev_aspects:
  cat = aspect.split('\t')[-1]
  word = aspect.split('\t')[2]
  dict_sents[word] = cat

In [ ]:
train_texts, train_ids = [], []
with open('/content/drive/MyDrive/more_train_split_reviews.txt') as f:
  lines = (line.rstrip() for line in f)
  lines = list(line for line in lines if line)
  for line in lines:
    text_id, text = line.split('\t')
    train_texts.append(text)
    train_ids.append(text_id)

In [ ]:
dev_texts, dev_ids = [], []
with open('/content/drive/MyDrive/dev_reviews.txt') as f:
  for line in f:
    text_id, text = line.rstrip('\r\n').split('\t')
    dev_texts.append(text)
    dev_ids.append(text_id)

In [ ]:
train_sentiment = []
with open('/content/drive/MyDrive/more_train_split_cats.txt') as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    train_sentiment.append(line)

In [ ]:
dev_sentiment = []
with open('/content/drive/MyDrive/dev_cats.txt') as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    dev_sentiment.append(line)

### Переразмечаем наши данные в формате BIO, где помимо тегов категории мы добавляем теги тональности

In [ ]:
reviews_train = {}
review_test = {}
for i in range(len(train_texts)):
  reviews_train[train_ids[i]] = train_texts[i]

for i in range(len(dev_texts)):
  review_test[dev_ids[i]] = dev_texts[i]

In [ ]:
aspects_train = defaultdict(list)
aspects_test = defaultdict(list)

In [ ]:
keys = ('category', 'mention', 'start', 'end', 'sentiment')
for t in train_aspects:
  text_id = t.split('\t')[0]
  line = t.split('\t')[1:]
  aspects_train[text_id].append(dict(zip(keys, line)))

for t in dev_aspects:
  text_id = t.split('\t')[0]
  line = t.split('\t')[1:]
  aspects_test[text_id].append(dict(zip(keys, line)))

In [ ]:
bio_train_texts = {}
for text_id, text in tqdm(list(reviews_train.items())):
  text_bio = []
  processed = nlp(text)
  for token in processed.iter_tokens():
    add = False
    for mention in aspects_train[text_id]:
      if token.start_char == int(mention['start']):
        bio = 'B-' + mention['category'] + '-' + mention['sentiment']
        add = True
        break
      elif token.end_char == int(mention['end']):
        bio = 'E-' + mention['category'] + '-' + mention['sentiment']
        add = True
        break
      elif int(mention['start']) < token.start_char < int(mention['end']):
        bio = 'I-' + mention['category'] + '-' + mention['sentiment']
        add = True
        break

    if not add:
      bio = 'O'

    text_bio.append(bio)

  bio_train_texts[text_id] = text_bio

  0%|          | 0/426 [00:00<?, ?it/s]

In [ ]:
bio_test_texts = {}
for text_id, text in tqdm(list(review_test.items())):
  text_bio = []
  processed = nlp(text)
  for token in processed.iter_tokens():
    add = False
    for mention in aspects_test[text_id]:
      if token.start_char == int(mention['start']):
        bio = 'B-' + mention['category'] + '-' + mention['sentiment']
        add = True
        break
      elif token.end_char == int(mention['end']):
        bio = 'E-' + mention['category'] + '-' + mention['sentiment']
        add = True
        break
      elif int(mention['start']) < token.start_char < int(mention['end']):
        bio = 'I-' + mention['category'] + '-' + mention['sentiment']
        add = True
        break

    if not add:
      bio = 'O'

    text_bio.append(bio)

  bio_test_texts[text_id] = text_bio

  0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
def word2features(sent, i):
    word = sent[i].text
    postag = sent[i].upos
    
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1].text
        postag1 = sent[i-1].pos
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1].text
        postag1 = sent[i+1].pos
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def text2features(text):
  prep_text = nlp(text)
  toks = []
  for sent in prep_text.sentences:
    for token in sent.words:
      toks.append(token)
  return [word2features(toks, i) for i in range(len(toks))]

def text2labels(text_ids, labels):
    return [labels[i] for i in text_ids]

def text2tokens(text):
    prep_text = nlp(text)
    toks = []
    for sent in prep_text.sentences:
      for token in sent.words:
        toks.append(token)
    return toks

In [ ]:
X_train = [text2features(s) for s in train_texts]
y_train = text2labels(train_ids, bio_train_texts)

X_test = [text2features(s) for s in dev_texts]
y_test = text2labels(dev_ids, bio_test_texts)

### Запускаем CRF

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.40922530878449587

In [ ]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                     precision    recall  f1-score   support

        B-Food-both      0.000     0.000     0.000        10
        E-Food-both      0.000     0.000     0.000         0
        I-Food-both      0.000     0.000     0.000         0
    B-Food-negative      0.467     0.101     0.167        69
    E-Food-negative      0.000     0.000     0.000        11
    I-Food-negative      0.000     0.000     0.000         6
     B-Food-neutral      0.292     0.160     0.207       119
     E-Food-neutral      0.167     0.057     0.085        35
     I-Food-neutral      0.091     0.029     0.044        34
    B-Food-positive      0.546     0.439     0.487       296
    E-Food-positive      0.476     0.319     0.382        94
    I-Food-positive      0.540     0.321     0.402       106
    B-Interior-both      0.000     0.000     0.000         3
    E-Interior-both      0.000     0.000     0.000         0
B-Interior-negative      0.000     0.000     0.000        12
E-Interior-negative    

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Food-both', 'E-Food-both', 'I-Food-both', 'B-Food-negative', 'E-Food-negative', 'I-Food-negative', 'B-Food-neutral', 'E-Food-neutral', 'I-Food-neutral', 'B-Food-positive', 'E-Food-positive', 'I-Food-positive', 'B-Interior-both', 'E-Interior-both', 'B-Interior-negative', 'E-Interior-negative', 'I-Interior-negative', 'B-Interior-neutral', 'E-Interior-neutral', 'I-Interior-neutral', 'B-Interior-positive', 'E-Interior-positive', 'I-Interior-positive', 'B-Price-both', 'B-Price-negative', 'E-Price-negative', 'I-Price-negative', 'B-Price-neutral', 'E-Price-neutral', 'I-Price-neutral', 'B-Price-positive', 'E-Price-positive', 'I-Price-positive', 'B-Service-both', 'I-Service-both', 'B-Service-negative', 'E-Service-negative', 'I-Service-negative', 'B-Service-neutral', 'E-Service-neutral', 'I-Service-neutral', 'B-Service-positive', 'E-Service-positive', 'I-Service-positive', 'B-Whole-both', 'E-Who

### Тем словам, которым CRF не присвоил категорию, но которые встречаются в обучающих и имеют там категорию, мы присваиваем ту же категорию и тональность, что и в обучающих данных. Результаты записываем в файл.

In [ ]:
with open('/content/drive/MyDrive/dev_pred_aspects_sent_crf.txt', "w") as f:
  for i in tqdm(range(len(dev_texts))):
    preds = y_pred[i]
    dev_text = nlp(dev_texts[i])
    dev = []
    dev_ends = []
    dev_starts = []
    text = nlp(dev_text)
    sents = text.sentences
    for sent in sents:
      toks = sent.tokens
      for tok in toks:
        dev.append(tok.text)
        dev_ends.append(tok.end_char)
        dev_starts.append(tok.start_char)
    
    y_2 = 0
    for y in range(len(preds)):
      if y == y_2:
        y_2 = y + 1
        if preds[y] != 'O':
          if 'positive' in preds[y]:
            sent = 'positive'
            cat = preds[y][2:-9]

          elif 'negative' in preds[y]:
            sent = 'negative'
            cat = preds[y][2:-9]
          
          elif 'neutral' in preds[y]:
            sent = 'neutral'
            cat = preds[y][2:-8]
          
          elif 'both' in preds[y]:
            sent = 'both'
            cat = preds[y][2:-5]
          
          
          id_text = dev_ids[i]
          tokens_span = dev[y]
          ends_span = dev_ends[y]
          starts_span = dev_starts[y]
          for e in range(y+1, len(preds)):
            if 'positive' in preds[y]:
              cat_e = preds[e][2:-9]

            elif 'negative' in preds[y]:
              cat_e = preds[e][2:-9]
            
            elif 'neutral' in preds[y]:
              cat_e = preds[e][2:-8]
            
            elif 'both' in preds[y]:
              cat_e = preds[e][2:-5]

            if cat_e != cat:
              break
            else:
              token = ' ' + dev[e]
              ends_span = dev_ends[e]
              tokens_span += token
              y_2 = e + 1

          print(id_text, cat, tokens_span, starts_span, ends_span, sent, sep="\t", file=f)

        elif preds[y] == 'O':
          if dev[y] in dict_sents.keys():
            id_text = dev_ids[i]
            cat = dict_words[dev[y]]
            sent = dict_sents[dev[y]]
            tokens_span = dev[y]
            ends_span = dev_ends[y]
            starts_span = dev_starts[y]
            print(id_text, cat, tokens_span, starts_span, ends_span, sent, sep="\t", file=f)

  0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
gold_aspect_cats = {}
with open('/content/drive/MyDrive/dev_aspects.txt') as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

### Оценка 1: accuracy по выделению упоминаний с категориями

In [ ]:
full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open('/content/drive/MyDrive/dev_pred_aspects_sent_crf.txt') as fp:
    for line in fp:    
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i], 
                            doc_gold_aspect_cats["ends"][i], 
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i], 
                                    doc_gold_aspect_cats["ends"][i], 
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break


In [ ]:
gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])

In [ ]:
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7144906743185079
Full match recall: 0.8272425249169435
Partial match ratio in pred: 0.8213773314203731
Full category accuracy: 0.7037302725968436
Partial category accuracy: 0.8070301291248206



### Оценка 2: accuracy по тональности упоминаний

In [ ]:
def sentiment_accuracy(matches):
    matched_sentiment = 0.
    for pair in matches:
        *_, gold_s = pair[0]
        *_, pred_s = pair[1]
        if gold_s == pred_s:
            matched_sentiment += 1
    print(f"Mention sentiment accuracy: {matched_sentiment / len(matches)}")

In [ ]:
sentiment_accuracy(fully_matched_pairs)

Mention sentiment accuracy: 0.786144578313253


In [ ]:
sentiment_accuracy(partially_matched_pairs)

Mention sentiment accuracy: 0.6375838926174496


### Оценка 3: accuracy по тональности категории

In [ ]:
def from_aspects_to_cats(path_from, path_to):
    aspects_d = {}
    with open (path_from, encoding='utf-8') as f:
        for line in f:
            line = line.split('\t')
            sent_id = line[0]
            cat = line[1]
            sent = line[-1].replace('\n', '')
            if sent_id not in aspects_d.keys():
                aspects_d[sent_id] = {}
            if cat not in aspects_d[sent_id]:
                aspects_d[sent_id][cat] = []
            aspects_d[sent_id][cat].append(sent)
    aspects_cat = dict.fromkeys(aspects_d.keys())
    cats = ['Food', 'Interior', 'Price', 'Whole', 'Service']
    for k in aspects_cat.keys():
        aspects_cat[k] = {}
        for c in cats:
            if c not in aspects_d[k].keys():
                aspects_cat[k][c] = 'absence'
            else:
                counter = Counter(aspects_d[k][c]).most_common()
                if len(counter)>1 and counter[0][0]==counter[1][0]:
                    aspects_cat[k][c] = 'both'
                else:
                    aspects_cat[k][c] = counter[0][0]
    with open(path_to, 'w', encoding='utf-8') as f:
        for k, v in aspects_cat.items():
            for c, s in v.items():
                print(k, c, s, sep='\t', file=f)

In [ ]:
from_aspects_to_cats('/content/drive/MyDrive/dev_pred_aspects_sent_crf.txt', '/content/drive/MyDrive/dev_pred_cats_sent.txt')

In [ ]:
with open('/content/drive/MyDrive/dev_cats.txt') as gc, open('/content/drive/MyDrive/dev_pred_cats_sent.txt') as pc:
    gold_labels = set(gc.readlines())
    pred_labels = set(pc.readlines())
    print(
        "Overall sentiment accuracy:",
        len(gold_labels & pred_labels) / len(gold_labels)
    )

Overall sentiment accuracy: 0.7126760563380282
